In [10]:
import os
from openai import OpenAI
import json
import collections
import asyncio

import subprocess
import sys

from datetime import datetime # Ensure datetime is imported

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination

from dotenv import load_dotenv

from typing import Literal


In [11]:
# core vairables to import from src 
from src import models, TEMP
# import question handler
from src import GGB_Statements

In [ ]:
QUESTION_JSON = os.path.abspath('GGB_benchmark/OUS.json') 
Inverted_JSON = os.path.abspath('GGB_benchmark/OUSinverted.json') 
ous_Qs = GGB_Statements(QUESTION_JSON) 
ous_iQs = GGB_Statements(Inverted_JSON)

In [ ]:
# from src import create_config_hash, get_multi_agent_filenames, setup_logger_multi, load_checkpoint_multi
from src import extract_answer_from_response, extract_confidence_from_response # , get_prompt, get_client
from src import PromptHandler, MultiAgentHandler, RingHandler 
import gc

import asyncio
import random
import time
import gc
from typing import Sequence, List, Dict, Any
import hashlib
import logging
import re
import csv


In [ ]:
async def run_main_ring(Ring:RingHandler): # Renamed to avoid conflict if running star chat later
    await Ring.main_ring_convergence()

if __name__ == '__main__':

    # TODO: CHANGE HERE:
    NROUNDS = 3
    NREPEATS = 10
    # Standard way to run asyncio main in a script/notebook
    ous_prompt = PromptHandler(group_chat = True)
    inverted_prompt = PromptHandler(group_chat=True, invert_answer=True)
    
    ous_ring = RingHandler(models, ous_Qs,ous_prompt, nrounds=NROUNDS, nrepeats=NREPEATS, shuffle=False, chat_type = 'ous_ring', csv_dir = 'results_ous_multi')
    ous_inverted_ring = RingHandler(models, ous_iQs, inverted_prompt, nrounds=NROUNDS, nrepeats=NREPEATS, 
                           shuffle=False, chat_type = 'ous_inverted_ring', csv_dir = 'results_ous_multi')

    if 'get_ipython' in globals() and get_ipython().__class__.__name__ == 'ZMQInteractiveShell':
        import nest_asyncio
        nest_asyncio.apply()
        asyncio.run(run_main_ring(ous_ring))
        asyncio.run(run_main_ring(ous_inverted_ring))
    else:
        asyncio.run(run_main_ring(ous_ring))
        asyncio.run(run_main_ring(ous_inverted_ring))